In [7]:
from qiskit_optimization.applications import Knapsack
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.utils import algorithm_globals
from qiskit.algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA

from qiskit_optimization.runtime import QAOAClient

from qiskit.primitives import Sampler

In [27]:
installation_costs = {
    'Supermarket_1': 30000,
    'Shopping_center_1': 35000,
    'Cinema': 32000,
    'University_1': 45000,
}

traffic_values = {
    'Supermarket_1': [0.06, 0.21, 0.86, 0.38, 0.16, 0.06, 0.38],
    'Shopping_center_1': [0.54, 0.59, 0.11, 0.29, 0.15, 0, 0],
    'Cinema': [0.40, 0.77, 0.99, 0.90, 0.84, 0.10, 0.58],
    'University_1': [0.03, 0.014, 0.86, 0.48, 0.71, 0.73, 0.99],
}

# Total available budget
budget = 75000

In [28]:
traffic_values_list = [sum(l)/len(l) for l in traffic_values.values()]
# demand_from_people = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

# values = list(map(lambda x: (x[0] + x[1]) / 2, list(zip(traffic_values_list, demand_from_people))))
installation_costs_list = list(installation_costs.values())


# Normalize
budget = int((budget / max(installation_costs_list))*100)
installation_costs_list = [int((x / max(installation_costs_list))*100) for x in installation_costs_list]
installation_costs_list

[66, 77, 71, 100]

In [29]:

prob = Knapsack(values=traffic_values_list, weights=installation_costs_list, max_weight=budget)
qp = prob.to_quadratic_program()
print(qp.prettyprint())

Problem name: Knapsack

Maximize
  0.30142857142857143*x_0 + 0.24*x_1 + 0.6542857142857142*x_2
  + 0.5448571428571428*x_3

Subject to
  Linear constraints (1)
    66*x_0 + 77*x_1 + 71*x_2 + 100*x_3 <= 166  'c0'

  Binary variables (4)
    x_0 x_1 x_2 x_3



In [30]:
# QAOA
meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, sampler=Sampler(), optimizer=COBYLA()))
result = meo.solve(qp)
print(result.prettyprint())
print("\nsolution:", prob.interpret(result))
print("\ntime:", result.min_eigen_solver_result.optimizer_time)

objective function value: 0.9557142857142857
variable values: x_0=1.0, x_1=0.0, x_2=1.0, x_3=0.0
status: SUCCESS

solution: [0, 2]

time: 16.528381824493408


In [31]:
from qiskit_optimization.converters import QuadraticProgramToQubo

# intermediate QUBO form of the optimization problem
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)
print(qubo.prettyprint())

Problem name: Knapsack

Minimize
  2.7405714285714287*c0@int_slack@0^2
  + 10.962285714285715*c0@int_slack@0*c0@int_slack@1
  + 21.92457142857143*c0@int_slack@0*c0@int_slack@2
  + 43.84914285714286*c0@int_slack@0*c0@int_slack@3
  + 87.69828571428572*c0@int_slack@0*c0@int_slack@4
  + 175.39657142857143*c0@int_slack@0*c0@int_slack@5
  + 350.79314285714287*c0@int_slack@0*c0@int_slack@6
  + 213.76457142857143*c0@int_slack@0*c0@int_slack@7
  + 10.962285714285715*c0@int_slack@1^2
  + 43.84914285714286*c0@int_slack@1*c0@int_slack@2
  + 87.69828571428572*c0@int_slack@1*c0@int_slack@3
  + 175.39657142857143*c0@int_slack@1*c0@int_slack@4
  + 350.79314285714287*c0@int_slack@1*c0@int_slack@5
  + 701.5862857142857*c0@int_slack@1*c0@int_slack@6
  + 427.52914285714286*c0@int_slack@1*c0@int_slack@7
  + 43.84914285714286*c0@int_slack@2^2
  + 175.39657142857143*c0@int_slack@2*c0@int_slack@3
  + 350.79314285714287*c0@int_slack@2*c0@int_slack@4
  + 701.5862857142857*c0@int_slack@2*c0@int_slack@5
  + 1403.

In [32]:
# qubit Hamiltonian and offset
op, offset = qubo.to_ising()
print(f"num qubits: {op.num_qubits}, offset: {offset}\n")
print(op)

num qubits: 12, offset: 37142.09428571427

-13384.800142857144 * IIIIIIIIIIIZ
- 15615.656000000003 * IIIIIIIIIIZI
- 14398.635142857143 * IIIIIIIIIZII
- 20279.956142857147 * IIIIIIIIZIII
- 202.80228571428577 * IIIIIIIZIIII
- 405.60457142857155 * IIIIIIZIIIII
- 811.2091428571431 * IIIIIZIIIIII
- 1622.4182857142862 * IIIIZIIIIIII
- 3244.8365714285724 * IIIZIIIIIIII
- 6489.673142857145 * IIZIIIIIIIII
- 12979.346285714288 * IZIIIIIIIIII
- 7909.289142857142 * ZIIIIIIIIIII
+ 6963.792 * IIIIIIIIIIZZ
+ 6421.158857142857 * IIIIIIIIIZIZ
+ 7491.352 * IIIIIIIIIZZI
+ 9043.885714285714 * IIIIIIIIZIIZ
+ 10551.2 * IIIIIIIIZIZI
+ 9729.028571428571 * IIIIIIIIZZII
+ 90.43885714285715 * IIIIIIIZIIIZ
+ 105.512 * IIIIIIIZIIZI
+ 97.29028571428572 * IIIIIIIZIZII
+ 137.02857142857144 * IIIIIIIZZIII
+ 180.8777142857143 * IIIIIIZIIIIZ
+ 211.024 * IIIIIIZIIIZI
+ 194.58057142857143 * IIIIIIZIIZII
+ 274.0571428571429 * IIIIIIZIZIII
+ 2.7405714285714287 * IIIIIIZZIIII
+ 361.7554285714286 * IIIIIZIIIIIZ
+ 422.048 * II